In [1]:
import pandas as pd
import os
from dotenv import load_dotenv
import mysql.connector

In [2]:
load_dotenv()

True

In [3]:
configuration = {
    'user' : os.getenv('user'),
    'password' : os.getenv('password'),
    'host' : os.getenv('host'),
    'database' : 'information_schema',
    'raise_on_warnings' : True
}

In [4]:
try:
    connection = mysql.connector.connect(**configuration)
    cursor = connection.cursor()

    cursor.execute("SELECT SCHEMA_NAME FROM SCHEMATA WHERE SCHEMA_NAME = 'renewable_power';")
    schema_exists = cursor.fetchone()

    if not schema_exists:
        cursor.execute("CREATE SCHEMA renewable_power;")
        print("Schema renewable_power created.")

        cursor.execute("USE renewable_power;")

    else:
        print("Schema already exists.")

    cursor.close()
    connection.close()

except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Something is wrong with your user name or password")
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("Database does not exist")
    else:
        print(err)
else:
    connection.close()

Schema renewable_power created.


In [5]:
try:
    connection = mysql.connector.connect(**configuration)
    cursor = connection.cursor()
    
    cursor.execute("""
                    SELECT EXISTS (
                    SELECT TABLE_NAME
                    FROM information_schema.TABLES
                    WHERE TABLE_SCHEMA LIKE 'renewable_power' AND TABLE_TYPE LIKE 'BASE TABLE' AND TABLE_NAME = 'times');
                """)
    table_exists = cursor.fetchone()[0]
    
    if not table_exists:
        cursor.execute("USE renewable_power;")
        cursor.execute("""
                    CREATE TABLE times (
                    time_id INT AUTO_INCREMENT PRIMARY KEY,
                    time DATETIME
                    );
                """)
        print("The 'times' table has been created.")
        
    else:
        print("The 'times' table already exists.")

    cursor.close()
    connection.close()

except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Something is wrong with your user name or password")
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("Database does not exist")
    else:
        print(err)
else:
    connection.close()

The 'times' table has been created.


In [6]:
try:
    connection = mysql.connector.connect(**configuration)
    cursor = connection.cursor()
    
    cursor.execute("""
                    SELECT EXISTS (
                    SELECT TABLE_NAME
                    FROM information_schema.TABLES
                    WHERE TABLE_SCHEMA LIKE 'renewable_power' AND TABLE_TYPE LIKE 'BASE TABLE' AND TABLE_NAME = 'weather_measurements');
                """)
    table_exists = cursor.fetchone()[0]
    
    if not table_exists:
        cursor.execute("USE renewable_power;")
        cursor.execute("""
                    CREATE TABLE weather_measurements (
                    measurement_id INT AUTO_INCREMENT PRIMARY KEY,
                    time_id INT,
                    FOREIGN KEY (time_id) REFERENCES times(time_id),
                    energy_delta INTEGER,
                    GHI DOUBLE,
                    temperature DOUBLE,
                    pressure INTEGER,
                    humidity INTEGER,
                    wind_speed DOUBLE,
                    rain_past_hour DOUBLE,
                    snow_past_hour DOUBLE,
                    clouds_all_day INTEGER,
                    is_sun INTEGER,
                    sunlight_time INTEGER,
                    day_length INTEGER,
                    sunlight_time_as_portion_of_day_length DOUBLE
                    );
                """)
        print("The 'weather_measurements' table has been created.")
        
    else:
        print("The 'weather_measurements' table already exists.")

    cursor.close()
    connection.close()

except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Something is wrong with your user name or password")
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("Database does not exist")
    else:
        print(err)
else:
    connection.close()

The 'weather_measurements' table has been created.


In [7]:
df_times = pd.read_csv('../1_Data/transformed_times_dataset.csv')

df_times.head()

,Time
0,2017-01-01 00:00:00
1,2017-01-01 00:15:00
2,2017-01-01 00:30:00
3,2017-01-01 00:45:00
4,2017-01-01 01:00:00


In [8]:
try:
    connection = mysql.connector.connect(**configuration)
    cursor = connection.cursor()

    cursor.execute("USE renewable_power;")
    
    insert_stmt = """
        INSERT INTO times (time)
        VALUES (%s)
    """

    # Iterate through the DataFrame and insert each row
    for _, row in df_times.iterrows():
        cursor.execute(insert_stmt, tuple(row))

    # Commit the transaction
    connection.commit()

    print("transformed_times_dataset.csv inserted into MySQL successfully.")

    # Getting time_id column for the weather_measurements table
    cursor.execute("SELECT time_id FROM times")
    time_ids = cursor.fetchall()
    
    cursor.close()
    connection.close()

except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Something is wrong with your user name or password")
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("Database does not exist")
    else:
        print(err)
else:
    connection.close()

transformed_times_dataset.csv inserted into MySQL successfully.


In [9]:
# Putting time_ids variable into a df for easy concat to weather_measurements df
df_time_ids = pd.DataFrame(time_ids, columns=['time_id'])

df_time_ids.head()

,time_id
0,1
1,2
2,3
3,4
4,5


In [10]:
df_weather_measurements = pd.read_csv('../1_Data/transformed_weather_measurements_dataset.csv')

df_weather_measurements.head()

,Energy delta[Wh],GHI,temp,pressure,humidity,wind_speed,rain_1h,snow_1h,clouds_all,isSun,sunlightTime,dayLength,SunlightTime/daylength
0,0,0.0,1.6,1021,100,4.9,0.0,0.0,100,0,0,450,0.0
1,0,0.0,1.6,1021,100,4.9,0.0,0.0,100,0,0,450,0.0
2,0,0.0,1.6,1021,100,4.9,0.0,0.0,100,0,0,450,0.0
3,0,0.0,1.6,1021,100,4.9,0.0,0.0,100,0,0,450,0.0
4,0,0.0,1.7,1020,100,5.2,0.0,0.0,100,0,0,450,0.0


In [11]:
# Recreate df with weather_measurements df and the time_ids all aligned
df_weather_measurements_with_time_id = pd.concat([df_time_ids, df_weather_measurements], axis=1)

df_weather_measurements_with_time_id.head()

,time_id,Energy delta[Wh],GHI,temp,pressure,humidity,wind_speed,rain_1h,snow_1h,clouds_all,isSun,sunlightTime,dayLength,SunlightTime/daylength
0,1,0,0.0,1.6,1021,100,4.9,0.0,0.0,100,0,0,450,0.0
1,2,0,0.0,1.6,1021,100,4.9,0.0,0.0,100,0,0,450,0.0
2,3,0,0.0,1.6,1021,100,4.9,0.0,0.0,100,0,0,450,0.0
3,4,0,0.0,1.6,1021,100,4.9,0.0,0.0,100,0,0,450,0.0
4,5,0,0.0,1.7,1020,100,5.2,0.0,0.0,100,0,0,450,0.0


In [12]:
try:
    connection = mysql.connector.connect(**configuration)
    cursor = connection.cursor()

    cursor.execute("USE renewable_power;")
    
    insert_stmt = """
        INSERT INTO weather_measurements (time_id, energy_delta, GHI, temperature, pressure, humidity, wind_speed, rain_past_hour, snow_past_hour, clouds_all_day, is_sun, sunlight_time, day_length, sunlight_time_as_portion_of_day_length)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """

    # Iterate through the DataFrame and insert each row
    for _, row in df_weather_measurements_with_time_id.iterrows():
        cursor.execute(insert_stmt, tuple(row))

    # Commit the transaction
    connection.commit()

    print("transformed_weather_measurements_dataset.csv inserted into MySQL successfully.")

    cursor.close()
    connection.close()

except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Something is wrong with your user name or password")
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("Database does not exist")
    else:
        print(err)
else:
    connection.close()

transformed_weather_measurements_dataset.csv inserted into MySQL successfully.
